# Getting Started with cuDF
#### By Yi Dong, Paul Hendricks
-------

While the world’s data doubles each year, CPU computing has hit a brick wall with the end of Moore’s law. For the same reasons, scientific computing and deep learning has turned to NVIDIA GPU acceleration, data analytics and machine learning where GPU acceleration is ideal. 

NVIDIA created RAPIDS – an open-source data analytics and machine learning acceleration platform that leverages GPUs to accelerate computations. RAPIDS is based on Python, has pandas-like and Scikit-Learn-like interfaces, is built on Apache Arrow in-memory data format, and can scale from 1 to multi-GPU to multi-nodes. RAPIDS integrates easily into the world’s most popular data science Python-based workflows. RAPIDS accelerates data science end-to-end – from data prep, to machine learning, to deep learning. And through Arrow, Spark users can easily move data into the RAPIDS platform for acceleration.

In this notebook, we will also show how to get started with GPU DataFrames using cuDF in RAPIDS.

**Table of Contents**

* Setup
* Loading data into a GPU DataFrame (GDF)
  * Loading data into a Pandas DataFrame
  * Converting a Pandas DataFrame to a GDF
* Working with the GDF
  * Take a look at the columns and their data types
  * Slice the GDF
  * Modify data types
  * Manipulate data with a user-defined function (UDF)
  * Sort the data
  * Filter the data
  * One-hot encode categorical columns
  * Split the data into training and validation sets
  * Turn the GDFs into matrices
* Conclusion

## Setup

This notebook was tested using the `nvcr.io/nvidia/rapidsai/rapidsai:0.5-cuda10.0-runtime-ubuntu18.04-gcc7-py3.7` Docker container from [NVIDIA GPU Cloud](https://ngc.nvidia.com) and run on the NVIDIA Tesla V100 GPU. Please be aware that your system may be different and you may need to modify the code or install packages to run the below examples. 

If you think you have found a bug or an error, please file an issue here: https://github.com/rapidsai/notebooks/issues

Before we begin, let's check out our hardware setup by running the `nvidia-smi` command.

In [ ]:
!nvidia-smi

Next, let's see what CUDA version we have:

In [ ]:
!nvcc --version

## Loading data into a GPU DataFrame (GDF)

### Loading data into a Pandas DataFrame

It's easy to load almost any sort of data (json, csv, etc) into a Pandas DataFrame. <br>
For example, let's import some census data from a compressed CSV file on disk:

In [ ]:
import pandas as pd; print('pandas Version:', pd.__version__)

In [ ]:
# read data from csv file into pandas dataframe
df = pd.read_csv('./data/ipums/ipums_easy.csv')

[Read more on using a Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)

### Converting a Pandas DataFrame to a GDF

Next, we use our `pandas.DataFrame` and to create a `cudf.dataframe.DataFrame` object using the `from_pandas` method.

In [ ]:
import cudf

# convert the Panda dataframe into a GPU dataframe
gdf = cudf.DataFrame.from_pandas(df)

And that's it! For the most part, working with GPU DataFrames will be the same as working with Pandas DataFrames. See the [cuDF documentation](https://cudf.readthedocs.io/en/latest/index.html) for more information.

## Working with the GDF

### Take a look at the columns and their data types

In [ ]:
# print the columns and their datatypes in this gdf
gdf.dtypes

### Slice the GDF

Woah! This GDF has a lot of columns, let's make it more manageable...

In [ ]:
# only select certain columns (and overwrite the gdf)
column_names = [
    'INCEARN', 'PERWT', 'ADJUST', 'STATEICP', 'ROOMS', 'BEDROOMS',
     'PHONE', 'VEHICLES', 'RACE', 'SEX', 'AGE', 'VETSTAT'
]
gdf = gdf.loc[:, column_names]

# show the first 5 records of each column
print(gdf.head(5))

### Modify data types

In [ ]:
gdf.dtypes

Looks like `INCEARN` and `PERWT` are integers when they should be floats. Let's fix that...

In [ ]:
import numpy as np

# convert the following two int64 columns to float64 data type
gdf['INCEARN'] = gdf['INCEARN'].astype(np.float64)
gdf['PERWT'] = gdf['PERWT'].astype(np.float64)

# take another look
gdf.dtypes

### Manipulate data with a user-defined function (UDF)

`INCEARN` is a column in our dataset that supposedly represents income earned; however, it does not truly represent the amount of income earned when adjusted for inflation. The `ADJUST` column represents the dollar inflation factor, which we can use to adjust `INCEARN` to the amount that the individual would have earned during the calender year. In our dataset, `ADJUST` is constant over all rows.

Below, we will define a simple function `adjust_incearn` that takes `INCEARN` and and multiplies it by a constant - in this case, the dollar inflation factor. We'll use the `applymap` method in our `cudf.dataframe.DataFrame` object to apply an element-wise function to transform the values in the Column.

In [ ]:
# define a function to adjust the incearn column
# so it more accurately represents income earned
adjust = gdf['ADJUST'][0]   # take constant from first row
print('adjustment factor: {}'.format(adjust))
def adjust_incearn(incearn):
    return adjust * incearn;

# apply it to the 'population' column
gdf['INCEARN'] = gdf['INCEARN'].applymap(adjust_incearn)

# drop the ADJUST column
gdf.drop_column('ADJUST')

# compute the mean
print('mean adjusted income: {}'.format(gdf['INCEARN'].mean()))

### Sort the data

Next, let's sort out data to do some light exploration.

In [ ]:
# sort the gdf by the INCEARN column
gdf = gdf.sort_values(by='INCEARN', ascending=True)
# reset the index so we can use loc slicing later
gdf = gdf.reset_index()
print(gdf.head(5))

Looks like we have some negative income values. Let's filter those out...

### Filter the data

We'll use the `query` method to filter our dataset. The `query` method takes as argument a boolean expression very similar to the `query` method for the `pandas.DataFrame` class. However, the `cudf.dataframe.DataFrame` implementation uses Numba to compile a GPU kernel. 

For more information on the syntax for arguments into `query`, see the Pandas documentation: 

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.query.html

In [ ]:
# how many records do we have?
print("{} = Original # of records".format(len(gdf)))

# filter out
gdf = gdf.query('INCEARN >= 0')

# how many records do we have left?
print("{} = New # of records".format(len(gdf)))

# sanity check...
print(gdf.head(5))

### One-hot encode categorical columns

Next, let's prepare our categorical columns. Machine learning models won't take as input strings as so we need to convert each column and its string representations to a numerical representation. The most common way to convert a Column with `n` elements and `k` unique categories to a numerical representation is to create a matrix of shape `n` by `k` and impute a 1 in cell `(i, j)` if the `ith` element is of category `j` and 0 otherwise, where $j \in k$. This is known as one-hot encoding.

In [ ]:
# define the categorical columns
cat_cols = set(['STATEICP', 'RACE', 'SEX', 'VETSTAT'])
# store the unique values for each category column
uniques = {}

# iterate through each categorical column and one-hot
# encode it using the unique values it has
for k in cat_cols:
    uniques[k] = gdf[k].unique_k(k=1000)
    cats = uniques[k][1:]  # drop first
    gdf = gdf.one_hot_encoding(k, prefix=k, cats=cats)
    del gdf[k]
    
# we should see many more columns since the categorical
# columns will get expanded due to one-hot encoding
gdf.dtypes

### Split the data into training and validation sets

Next, let's split out data into an 80% train dataset and a 20% validation dataset.

In [ ]:
# enforce float64 data type on ALL columns
for k in gdf.columns:
    gdf[k] = gdf[k].astype(np.float64)

# set the fractions for training and validation
fractions = {
    "train": 0.8,
    "valid": 0.2
}

# validation splitpoint
splitpoint = int(len(gdf) * fractions["train"])
print('splitpoint: {} of {} is {}'.format(fractions["train"], len(gdf), splitpoint))

# break the gdf up into training and validation sets
gdfs = {
    "train": gdf.loc[:splitpoint],
    "valid": gdf.loc[splitpoint:]
}
print('gdfs["train"] has {} rows'.format(len(gdfs["train"])))
print('gdfs["valid"] has {} rows'.format(len(gdfs["valid"])))

### Turn the GDFs into matrices

Lastly, we want to convert our GPU DataFrame to a GPU Matrix for usage as input to other machine learning libraries such as cuML and XGBoost. We can use the `as_gpu_matrix` method to facillitate this conversion.

In [ ]:
# produce gpu matrices (to input to ML libraries, etc.)
# this step should not be necessary in the near future
# (should be able to use gdf as input)
matrices = {
    "train": {
        "x": gdfs["train"].as_gpu_matrix(columns=gdf.columns[1:]),
        "y": gdfs["train"].as_gpu_matrix(columns=[gdf.columns[0]])
    },
    "valid": {
        "x": gdfs["valid"].as_gpu_matrix(columns=gdf.columns[1:]),
        "y": gdfs["valid"].as_gpu_matrix(columns=[gdf.columns[0]])
    }
}

# check the matrix shapes (sanity check)
print('matrices["train"]["x"] shape:', matrices["train"]["x"].shape)
print('matrices["train"]["y"] shape:', matrices["train"]["y"].shape)
print('matrices["valid"]["x"] shape:', matrices["valid"]["x"].shape)
print('matrices["valid"]["y"] shape:', matrices["valid"]["y"].shape)

## Conclusion

To learn more about RAPIDS, be sure to check out: 

* [Open Source Website](http://rapids.ai)
* [GitHub](https://github.com/rapidsai/)
* [Press Release](https://nvidianews.nvidia.com/news/nvidia-introduces-rapids-open-source-gpu-acceleration-platform-for-large-scale-data-analytics-and-machine-learning)
* [NVIDIA Blog](https://blogs.nvidia.com/blog/2018/10/10/rapids-data-science-open-source-community/)
* [Developer Blog](https://devblogs.nvidia.com/gpu-accelerated-analytics-rapids/)
* [NVIDIA Data Science Webpage](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/)